In [41]:
!ollama pull llama3.2

from openai import OpenAI
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "What is 2 + 2?"}]
)

print(response.choices[0].message.content)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 
2 + 2 = 4.


In [42]:
message = "Estoy duro"

messages = [
    {"role": "user", "content": message}
]

messages

[{'role': 'user', 'content': 'Estoy duro'}]

In [43]:
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = openai.chat.completions.create(
 model=MODEL,
 messages=messages
)
response.choices[0].message.content

'Parece que estás expresando un estado físico o emocional. ¿Quieres hablar sobre lo que te está pasando? Estoy aquí para escucharte y ofrecerte apoyo.'

In [44]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    I realize this is inefficient as we're parsing twice! This is to keep the code in the lab simple.
    Feel free to use a class and optimize it!
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]

ed = fetch_website_contents("https://edwarddonner.com")
print (ed)

Home - Edward Donner

Home
AI Curriculum
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
If left unchecked, I will happily drone on about LLMs to anyone that will listen. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To my total joy (and shock) they’ve become best-selling, top-rated courses, with 400,000 enrolled across 190 countries. 

In [45]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.

"""

In [46]:

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [47]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [48]:
messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nAI Curriculum\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to 

In [49]:
def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [50]:
summarize("https://edwarddonner.com")

'Looks like this website is run by Ed, a wannabe AI guru, aka a person who really loves talking about LLMs. On his website, he has a bunch of courses and resources related to AI, but also shares updates about his life as the CTO (or maybe CEO?) of Nebula.io, an AI startup that sounds like it\'s trying to make making you discover your potential and reason for being a thing, whatever that means.\n\nAnyway, Ed seems to have some news, too. There\'s been a few major announcements on what he calls his "neural network hacker channel" (nerd alert!) - including some n8n-related builder stuff and an AI engineering track for building agents and voice agents because why not?'

In [54]:
from IPython.display import Markdown, display
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:

display_summary("https://cnn.com") 

In [57]:
system_prompt = """
You are a deal finder assistant that analyzes the contents of a website,
and provides a summary of all the deals of the day, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.

""" 

user_prompt = "Here are the contents of a website. Provide a short summary of this website. If it includes news or announcements, then summarize these too."


messages =[
    {"role": "system" , "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = openai.chat.completions.create(
 model=MODEL,
 messages=messages
)
print(response.choices[0].message.content)

# Website Summary
### Overview
The website appears to be an e-commerce platform offering discounts and promotions on various products from multiple categories.

### Deals of the Day

*   **50% off** on all outdoor gear at REI, including camping equipment and clothing.
*   **Buy one get one free** on all home decor items at Wayfair.
*   **Discounts up to 75% off** on designer clothing at Saks Off 5th.

### News & Announcements

*   *Temporary Closure*: Due to staffing issues, the website will be closed on Friday, March 19th until Monday, March 22nd. No refunds or cancellations of orders placed during this time.
*
